In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.0
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkContext available as sc, HiveContext available as sqlContext.


In [5]:
#从hdfs读取文件
train= sc.textFile("hdfs:///user/train.csv")
test = sc.textFile("hdfs:///user/test.csv")
print train.count(),test.count()

398062 85443


In [11]:
#将数据转成LabeledPoint格式
from pyspark.mllib.regression import LabeledPoint
def parsePoint(line):
    values = [float(x.strip()) for x in line.split(',')]
    return LabeledPoint(values[-1],values[:30])

In [12]:
train_parsed = train.map(parsePoint)
test_parsed = test.map(parsePoint)

In [14]:
#训练决策树模型
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
# Train a DecisionTree model.
model = DecisionTree.trainClassifier(train_parsed, numClasses=2, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)

In [19]:
#打印模型信息
print('Learned classification tree model:')
print(model.toDebugString())

Learned classification tree model:
DecisionTreeModel classifier of depth 5 with 57 nodes
  If (feature 13 <= -1.88824809623)
   If (feature 3 <= 0.0725116860673)
    If (feature 9 <= -2.72197181022)
     If (feature 11 <= -1.78485905966)
      If (feature 1 <= 0.985515455055)
       Predict: 0.0
      Else (feature 1 > 0.985515455055)
       Predict: 1.0
     Else (feature 11 > -1.78485905966)
      Predict: 0.0
    Else (feature 9 > -2.72197181022)
     If (feature 0 <= 1.45758964351)
      Predict: 0.0
     Else (feature 0 > 1.45758964351)
      If (feature 25 <= -0.184385768522)
       Predict: 1.0
      Else (feature 25 > -0.184385768522)
       Predict: 0.0
   Else (feature 3 > 0.0725116860673)
    If (feature 0 <= 1.97389157471)
     If (feature 9 <= 1.08317963167)
      If (feature 11 <= -1.78485905966)
       Predict: 1.0
      Else (feature 11 > -1.78485905966)
       Predict: 1.0
     Else (feature 9 > 1.08317963167)
      Predict: 0.0
    Else (feature 0 > 1.97389157471)
   

In [30]:
#计算测试集误差
predictions = model.predict(test_parsed.map(lambda x: x.features))
PredictionsAndLabels = predictions.zip(test_parsed.map(lambda lp: lp.label))
testErr = PredictionsAndLabels.filter(lambda (v, p): v != p).count() / float(test_parsed.count())
print('Test Error = ' + str(testErr))

Test Error = 0.0538487646735


In [32]:
#计算AUC值
from pyspark.mllib.evaluation import BinaryClassificationMetrics

# Instantiate metrics object
metrics = BinaryClassificationMetrics(PredictionsAndLabels)

# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)

Area under ROC = 0.932221133739
